# Imports

In [36]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Train

In [4]:
train_set = pd.read_csv('train/train_encoded.csv')

In [23]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Ensamble

In [33]:
model1 = KNeighborsClassifier(n_neighbors=17, weights='distance', metric='manhattan')
model2 = CatBoostClassifier(silent=True)
model3 = XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.7,\
                           learning_rate = 0.1, max_depth = 37, n_estimators = 60)
model4 = LGBMClassifier(max_depth=20, metric='binary_logloss', learning_rate=0.1)
model5 = LGBMClassifier()
model6 = KNeighborsClassifier(n_neighbors=49, weights='distance', metric='manhattan')
model7 = CatBoostClassifier(silent=True,learning_rate=0.1, subsample=0.6, n_estimators=57, random_state=42)
model8 = RandomForestClassifier(n_estimators=100, min_samples_split=5, min_samples_leaf=20, max_depth=70)
model9 = RandomForestClassifier(n_estimators=57, min_samples_split=4, max_depth=30, min_samples_leaf=5,\
                                random_state=111)

In [34]:
estimators = [('knn17', model1), ('catboost', model2), ('xgb', model3), ('lgbm', model4), ('lgbm2', model5),\
             ('knn49', model6), ('catboost_2000', model7), ('rf100', model8), ('rf57', model9)]
model = VotingClassifier(estimators=estimators, voting='hard')

model.fit(X_train, y_train)

VotingClassifier(estimators=[('knn17',
                              KNeighborsClassifier(metric='manhattan',
                                                   n_neighbors=17,
                                                   weights='distance')),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f9684e150d0>),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.7, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            inte...
                                            reg_lambda=None,
                                            scale_pos_weight=None,
                              

In [35]:
preds = model.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score %f" % (f1))

F1-Mean Score 0.818444


# Test

In [60]:
test_set = pd.read_csv('test/test_encoded.csv')

In [61]:
col_id = test_set['id']
test_set.drop('id', axis=1, inplace=True)

In [62]:
preds = model.predict(test_set)
preds

array([0, 1, 1, ..., 1, 1, 1])

In [63]:
test_set['id'] = col_id

In [64]:
test_set['target'] = preds.astype('int64')
test_set[['id', 'target']]

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [65]:
test_set[['id', 'target']].to_csv('result.csv', header=True, index=False)